## Introduction to FAISS (Pinecone/James Briggs course)

In [1]:
import requests
import os

In [2]:
data_url = "https://raw.githubusercontent.com/jamescalam/data/main/sentence_embeddings_15K/"

In [ ]:
if not os.path.exists('../../data/data_faiss_intro'):
    os.mkdir('../../data/data_faiss_intro')

for i in range(57):
#     if i < 10:
#         i = '0' + str(i)
    res = requests.get(data_url+f"embeddings_{i}.npy")
    with open(f'../../data/data_faiss_intro/embeddings_{i}.npy', 'wb') as fp:
        for chunk in res:
            fp.write(chunk)
    print('.', end='')

In [ ]:
res = requests.get(f"{data_url}sentences.txt")
with open(f"../../data/data_faiss_intro/sentences.txt", 'wb') as fp:
    for chunk in res:
        fp.write(chunk)

In [3]:
import numpy as np

In [4]:
for i  in range(57):
    if i == 0:
        with open(f'../../data/data_faiss_intro/embeddings_{i}.npy', 'rb') as fp:
            sentence_embedding = np.load(fp)
    else:
        with open(f'../../data/data_faiss_intro/embeddings_{i}.npy', 'rb') as fp:
            sentence_embedding = np.append(
                sentence_embedding, 
                np.load(fp), axis=0
            )      

In [5]:
sentence_embedding.shape

(14504, 768)

In [6]:
with open('../../data/data_faiss_intro/sentences.txt', 'r') as fp:
    lines = fp.read().split('\n')

In [7]:
lines[:5]

['A group of four children dancing in a backyard.',
 'The Conference Board said its measure of business confidence, which had fallen to 53 in the first quarter of 2003, improved to 60 in the most recent second quarter.',
 'a person eating a meal, often in a restaurant',
 'When you crossed the line, you violated the constitutional right," said Charles Weisselberg, who teaches law at the University of California, Berkeley.',
 "Ross Garber, Rowland's legal counsel, said the governor would have no comment on the condo deal."]

In [8]:
from sentence_transformers import SentenceTransformer

In [9]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [12]:
# model.encode(['Hello wolrd'], device='cuda', show_progress_bar=True)

In [ ]:
import faiss

In [14]:
#IndexFlatL2 (eulidean distance)
d = sentence_embedding.shape[1]
index = faiss.IndexFlatL2(d)

In [16]:
index.is_trainedained

True

In [17]:
index.add(sentence_embedding)

In [18]:
index.ntotal

14504

In [19]:
xq = model.encode(['someone sprints with a football'], device = 'cuda')

In [20]:
k = 4 # similar vectors number

In [21]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 4586 10252 12465   190]]
CPU times: user 122 ms, sys: 0 ns, total: 122 ms
Wall time: 164 ms


In [23]:
[f'{i}: {lines[i]}' for i in I[0]]

['4586: A group of football players is running in the field',
 '10252: A group of people playing football is running in the field',
 '12465: Two groups of people are playing football',
 '190: A person playing football is running past an official carrying a football']